<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/10f_Mutli_step_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-step CNNs**

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.vis_utils import plot_model

With multi-step CNNs the data inputs and outputs will be multiple time steps

For example:<br>
Given:<br>
[10,20,30,40,50,60,70,80,90]<br>

Three time steps can be the input<br>
[10,20,30]
These are used to predict the 2 time step output<br>
[40,50]

**Define the split_sequence for multi-step forecasting**

In [ ]:
def split_sequence(sequence, n_steps_in, n_steps_out): 
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    # check if we are beyond the sequence 
    if out_end_ix > len(sequence):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix] 
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

**Create a time series**

In [ ]:
raw_seq = [15,25,35,45,55,65,75,85,95,105]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# summarize the data
for i in range(len(X)):
  print(X[i], y[i])

In [ ]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

**Create the model**

In [ ]:
model = Sequential()
model.add(Conv1D(24, 2, activation='relu', input_shape=(n_steps_in, n_features))) 
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(18, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

**Train the model**

In [ ]:
model.fit(X, y, epochs=2000, verbose=0)

**Make a prediction**<br>
Expecting 105, 115

In [ ]:
x_input = array([75, 85, 95])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

**Assignment:**<br>
1. Modify the number of neurons in the model to improve the prediction. <br>
2. Then try predicting time steps further out.
